# DAT257x: Reinforcement Learning Explained

## Lab 4: Dynamic Programming

### Exercise 4.3 Policy Iteration

Policy Iteration calculates the optimal policy for an MDP, given its full definition.  The full definition of an MDP is the set of states, the set of available actions for each state, the set of rewards, the discount factor, and the state/reward transition function.

In [22]:
import test_dp               # required for testing and grading your code
import gridworld_mdp as gw   # defines the MDP for a 4x4 gridworld

**Implement the algorithm for Policy Iteration**.  Policy Iteration calculates the optimal policy for an MDP by doing repeated steps of policy evaluation and policy improvement.

A empty function **policy_iteration** is provided below; implement the body of the function to correctly calculate the optimal policy for an MDP.  The function defines 5 parameters - a definition of each parameter is given in the comment block for the function.  For sample parameter values, see the calling code in the cell following the function.

Note that there is a subtle difference between the algorithm for Policy Evaluation, which assumes the policy is stochastic, and the Policy Evaluation step for the Policy Iteration algorithm, which assumes the policy is deterministic.  This means that you cannot directly call your previous code, but you can reuse large pieces of it for the Policy Evaluation step.

In [23]:
def policy_eval(V, state_count, gamma, theta, policy, get_transitions):
        
    while True:
        
        delta = 0
        
        for i in range(state_count):
            
            v = V[i]
            
            transitions = gw.get_transitions(state=i, action=policy[i])
            
            inner_sum = 0
            
            for (trans) in transitions:
                
                next_state, reward, probability = trans    # unpack tuple
                
                # Calculate sigma(r, sprime) * p(r, sprime | s, a)
                    
                inner_sum = inner_sum + probability * (reward + gamma * V[next_state])
                
            V[i] = inner_sum
            
            delta = max(delta, abs(v - V[i]))
            
        # check exit condition

        if delta < theta:
            
            break
    
    return V

In [25]:
def policy_iteration(state_count, gamma, theta, get_available_actions, get_transitions):
    """
    This function computes the optimal value function and policy for the specified MDP, using the Policy Iteration algorithm.
    'state_count' is the total number of states in the MDP. States are represented as 0-relative numbers.
    
    'gamma' is the MDP discount factor for rewards.
    
    'theta' is the small number threshold to signal convergence of the value function (see Iterative Policy Evaluation algorithm).
    
    'get_available_actions' returns a list of the MDP available actions for the specified state parameter.
    
    'get_transitions' is the MDP state / reward transiton function.  It accepts two parameters, state and action, and returns
        a list of tuples, where each tuple is of the form: (next_state, reward, probabiliity).  
        
    """
    V = state_count*[0]                # init all state value estimates to 0
    pi = state_count*[0]
    
    # init with a policy with first avail action for each state (arbitrary)
    
    for s in range(state_count):
        
        avail_actions = get_available_actions(s)
        pi[s] = avail_actions[0]
        
    # insert code here to iterate using policy evaluation and policy improvement (see Policy Iteration algorithm)
    
    policy_stable = True
    
    while True:
        
       # print("policy_stable = ", policy_stable)
        
        # Get the value of each state given current policy pi
        
        V = policy_eval(V, state_count, gamma, theta, pi, get_transitions)
          
        # loop over each state
        
        for i in range(state_count):
            
            # store old action pi[i]
            
            old_action = pi[i]
            
            # For this state need to get all possible actions to check if there is a 'better' action
            
            action_probs = get_available_actions(i)
            
            action_vals = [] # array to store action value pairs
            
            # Loop over all possible action
            
            for action in action_probs:
                
                # Get possible transitions for this action
                
                transitions = gw.get_transitions(state=i, action=action)
                
                # Calculate sigma(r, sprime) * p(r, sprime | s, a)
                
                inner_sum = 0
                
                for (trans) in transitions:
                    
                    next_state, reward, probability = trans    # unpack tuple
                    
                    inner_sum = inner_sum + probability * (reward + gamma * V[next_state])
                    
                action_vals.append((action, inner_sum)) # store action value pair
                
            action_vals = sorted(action_vals, key = lambda x: x[1], reverse = True) # sort in descending order
            
            # replace current policy action at current state with new 'better' action
            
            pi[i] = action_vals[0][0]
            
            if action_vals[0][0] != old_action: # if there is a better action, set policy_stable to False
                
                policy_stable = False
                
        if policy_stable == True: # if no better action was found, break
            
            break
            
        else:
            
            policy_stable = True
    
    return (V, pi)        # return both the final value function and the final policy

First, test our function using the MDP defined by gw.* functions.

In [26]:
n_states = gw.get_state_count()

# test our function
values, policy = policy_iteration(state_count=n_states, gamma=.9, theta=.001, get_available_actions=gw.get_available_actions, \
    get_transitions=gw.get_transitions)

print("Values=", values)
print("Policy=", policy)

Values= [0.0, -1.0, -1.9, -2.71, -1.0, -1.9, -2.71, -1.9, -1.9, -2.71, -1.9, -1.0, -2.71, -1.9, -1.0]
Policy= ['up', 'left', 'left', 'down', 'up', 'up', 'up', 'down', 'up', 'up', 'down', 'down', 'up', 'right', 'right']


**Expected output from running above cell:**

`
Values= [0.0, -1.0, -1.9, -2.71, -1.0, -1.9, -2.71, -1.9, -1.9, -2.71, -1.9, -1.0, -2.71, -1.9, -1.0]
Policy= ['up', 'left', 'left', 'down', 'up', 'up', 'up', 'down', 'up', 'up', 'down', 'down', 'up', 'right', 'right']
`

In [27]:
import numpy as np
a = np.append(values, 0)
np.reshape(a, (4,4))

array([[ 0.  , -1.  , -1.9 , -2.71],
       [-1.  , -1.9 , -2.71, -1.9 ],
       [-1.9 , -2.71, -1.9 , -1.  ],
       [-2.71, -1.9 , -1.  ,  0.  ]])

Now, test our function using the test_dp helper.  The helper also uses the gw MDP, but with a different gamma value.
If our function passes all tests, a passcode will be printed.

In [28]:
a = np.append(policy, policy[0])
np.reshape(a, (4,4))

array([['up', 'left', 'left', 'down'],
       ['up', 'up', 'up', 'down'],
       ['up', 'up', 'down', 'down'],
       ['up', 'right', 'right', 'up']], dtype='<U5')

In [29]:
# test our function using the test_db helper
test_dp.policy_iteration_test( policy_iteration ) 


Testing: Policy Iteration
passed test: return value is tuple
passed test: length of tuple = 2
passed test: v is list of length=15
passed test: values of v elements
passed test: pi is list of length=15
passed test: values of pi elements
PASSED: Policy Iteration passcode = 9970-010
